In [ ]:
#%run /home/daniufundar/Documents/Fundar/indice-mercado-trabajo-ingresos/modulos/funciones/01_funciones.ipynb
%run /Users/danielarisaro/Documents/Fundar/indice-mercado-trabajo-ingresos/modulos/funciones/01_funciones.ipynb

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Definimos path
pathdata = '/home/daniu/Documentos/fundar/indice-mercado-trabajo-ingresos/'
pathdata = '/home/daniufundar/Documents/Fundar/indice-mercado-trabajo-ingresos/'
pathdata = '/Users/danielarisaro/Documents/Fundar/indice-mercado-trabajo-ingresos/'

In [ ]:
df_people_pool = pd.read_csv(pathdata + 'data_output/Base_pool_individuos_solo_con_replicas_actuales.csv', low_memory=False, index_col=0)
df_houses_pool = pd.read_csv(pathdata + 'data_output/Base_pool_hogares_solo_con_replicas_actuales.csv', low_memory=False, index_col=0)

In [ ]:
tabla_pob_ocupada_asalariada = calcular_tabla_poblacion_ocupada_asalariada(df_people_pool, tipo='Urbano', base='Individual')

In [ ]:

tasa, size, ratio, error = ratio_tasa_inactividad(df_people_pool, tipo='Urbano', base='Individual', confidence_level=0.95)
ratio_inactividad = pd.concat([tasa, size, ratio, error], axis=1)

tasa, size, ratio, error = ratio_tasas_empleo(df_people_pool, tipo='Urbano', base='Individual', confidence_level=0.95, pool=True)
empleo = pd.concat([tasa, size, ratio, error], axis=1)

tasa, size, ratio, error = ratio_tasas_desempleo(df_people_pool, tipo='Urbano', base='Individual', confidence_level=0.95, pool=True)
desempleo = pd.concat([tasa, size, ratio, error], axis=1)

tasa, size, ratio, error = acceso_aguinaldo(df_people_pool, tipo='Urbano', base='Individual', confidence_level=0.95)
empleo_asalariado_con_aguinaldo = pd.concat([tasa, size, ratio, error], axis=1)

tasa, size, ratio, error = acceso_obra_social(df_people_pool, tipo='Urbano', base='Individual', confidence_level=0.95)
empleo_asalariado_con_obra_social = pd.concat([tasa, size, ratio, error], axis=1)

In [ ]:

tasa, size, ratio, error = horas_mediana_remuneradas(df_people_pool, tipo='Urbano', base='Individual', confidence_level=0.95, pool=True)
horas_remuneradas_mediana = pd.concat([tasa, size, ratio, error], axis=1)